## Korpushersteller

GitHub Repository clonen und in GoogleColab anlegen

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/edadunashvili/ThePrax.git

In [ ]:
cd ThePrax

In [ ]:
!pwd

Ein Typ bestimmen

In [1]:
typNum=''
episode_string_train = "gesamt_string_train.csv"
episode_roh_train = "gesamt_roh_train.csv"

Alte Datei mit gleichem Namen löschen

In [2]:
import os
if os.path.exists(episode_string_train):
    os.remove(episode_string_train)
else:
    print("Diese Datei existiert nicht")

Diese Datei existiert nicht


Nach den entsprechenden Textdateien im Ordnet suchen und in einer rohe Datei zusammentragen

In [3]:
import glob

def word_to_lex(word):
    ret=(word) 
    return ret

def write_back(words):
    with open(episode_roh_train,"a", encoding='utf-8') as output:
        for word in words:
            #print(word)
            as_lex = word_to_lex(word[0])
            full_word = '"' + as_lex + '"'
            for sub_word in word[1:]:
                full_word += " , "  '"' + sub_word + '"'
            full_word +="\n"
            output.write(full_word)

def clean(line):
    line = line.replace("\n"," ").strip().lower()
    line = line.replace("ä","ae").replace("ü","ue").replace("ö","oe").replace("ß","ss").replace(",","").replace("«","")
    line = line.replace("»","").replace(".","").replace(":","").replace(";","").replace('"',"")
    line = line.replace("?","").replace("!","").replace("á","a").replace(",","").replace("\t"," ").replace("'","")
    line = line.replace("‹","").replace("›","").replace("-"," ").replace("'('","").replace("')'","").replace('>','')
    line = line.replace("    "," ").replace("   "," ").replace("  "," ").replace('–','').replace('—','').replace('<','')
    line = line.replace("Â", "A").replace("ø", "oe").replace('“','').replace('„','').replace('(','').replace(')','')
    line = line.replace("‚", "").replace(']','').replace('[','')
    if line == "": 
        return
    
    line=line.split("|")
    line[0]=line[0].split("|")[0]
    flex=[]
    try:
        flex=line[1].split("")
    except:
        pass
    value=str(line)
    line=str(line)   
    flex.append(line)
    ret=[]
    for i in flex:
        ret.append((i,value[0]))
    return ret

with open(episode_roh_train, "w", encoding='utf-8') as output:
    output.write ("quelle,episode,index_string,index_binar\n")
pairs = []

for file in glob.glob("Trainingsdaten/*.txt"):
    if typNum in file:
        with open(file, 'r', encoding='utf-8') as episode:
            for line in episode.readlines():
                clean_words = clean(line)
                pairs = pairs + clean_words
write_back(pairs)

Rohdatei endgültig überarbeiten und in eine CSV Datei speichern

In [4]:
fin = open(episode_roh_train,'r', encoding ='utf-8')
fout = open(episode_string_train, "wt", encoding ='utf-8')
for kfz in fin:
    fout.write(kfz.replace(', "["',"").replace('"[', "").replace(']"',"").replace("', '", "','").replace(" '", "'"))                      
fin.close()
fout.close()

Liste von einmaligen Episoden erstellen in der CSV Datei

In [5]:
import pandas as pd
df = pd.read_csv(episode_string_train, encoding='utf-8')
from collections import Counter
indexliste=Counter(df.index_string)
print(*indexliste, sep='\n')

'e301_a_'
'e300_c_anfangssituation_ankunft_und_erkundigung_der_not'
'e300_e_vorfeld_des_kampfes_bis_zum_sieg'
'e300_i_rueckkehr_der_koenigstochter_und_die_suche_nach_dem_held retter'
'e300_o_auftritt_des_helden_vor_dem_koenig_bis_zum_ende_der_geschichte'
'e554_a_'
'eamom_a_'
'e516_a_'
'e513a_a_'
'e302_a_'
'e300_g_nach_dem_sieg_bis_zur_trennung_des_befreiers_und_der_befreiten'
'e300_k_misshandlung_des_falschen_helden_bis_zur_hochzeitstag'
'e328_a_'
'ecom_a_'
'eundf_a_'
'e300_m_rueckkehr_des_helden_ _von_der_erkundung_ueber_die_hochzeit_bis_zum_wiedergewinn_der_aufmerksamkeit'
'e303_g_erstes_treffen_mit_der_hexen'
'e303_i_vom_erkundigung_der_not_bis_zum_aufbruch_des_helden_zur_hexe'
'e303_k_vom_afbruch_zur_hexe_bis_zur_ihren_ueberweltigung'
'e303_c_eingangssituation_bis_zur_trennung'
'e314_a_'
'e850_a_'
'e303_m_eifersucht'
'e315_a_'
'e303_q_rueckkehr_und_finale'
'e303_o_erloesung_der_hexe'
'e567_a_'
'e562_a_'
'e550_a_'
'e590_a_'
'e303_e_ankommen_an_den_fremden_ort_und_eheschliessung'
'e5

Liste von allen Episoden in der CSV Datei

In [6]:
#import pandas as pd
#df = pd.read_csv(episode_string_train, encoding='utf-8')
#x='_'
#for gesep in (df.index_string):
    #if x in gesep:
        #print(gesep) 

## Auswerter

Elementenstruktur definieren: vorherige Episode - gesuchte Episode - folgende Episode

In [8]:
import pandas as pd
df = pd.read_csv('gesamt_string_train.csv', encoding='utf-8')
from collections import Counter

def ep_format(ep_full):
    return (ep_full.split('_'))[0]

def quellenvergleich (df, i1, i2):
#     print(df.quelle[i1],df.quelle[i2])
    return df.quelle[i1]==df.quelle[i2]

def ast(gesep, df):
    ep_tree = {}
    a_liste = []
    z_liste = []  
    df_len = len(df.index_string)
    for i, ep in enumerate(df.index_string):
        if gesep == ep:
             #print(i, ep_full)
            if (i > 0)&(quellenvergleich(df, i, i-1)):
                a = df.index_string[i-1]
            else:
                a = 'eAnfang_a_'
            if (i < df_len - 1):
                if not (quellenvergleich(df, i, i+1)):
                    z = 'eEnde_a_'
                else:        
                    z = df.index_string[i+1]
            else:
                z = 'eEnde_a_'
            a_liste.append(a)
            z_liste.append(z)
    return {gesep: [Counter(a_liste), Counter(z_liste)]}

Alle im Korpus vorhandene Episoden extrahieren

In [9]:
def alle_aeste(gesep, df):
    episoden_baeume = {}
    
    ep_list = []
    for ep_full in df.index_string:  
        
        ep = ep_format(ep_full)      
        if gesep == ep:          
            
            ep_list.append(ep_full)
    for ep in set(ep_list):
        episoden_baeume.update(ast(ep,df))
    return episoden_baeume

Episoden die gesuchte Episode umlegen und in die Liste in die richrige Reihenfolge darstellen

In [10]:
def key_val_printer(d):
    for k, v in d.items():
        print(v, k, sep=':')
    #print()
        
def baum_printer(baeume: dict, baum: str):
    key_val_printer (baeume[baum][0])
    
    
    print(80*'-')
    print(sum(baeume[baum][0].values()), baum)
    print(80*'-')
    key_val_printer(baeume[baum][1]) 
    
            
def wald_printer(wald: dict):
    for baum in sorted(wald.keys()):
        baum_printer (wald, baum)
        print('\n',80*'=','\n')

Die gesuchte Episode bestimmen

In [20]:
synop = alle_aeste("'e300", df)
wald_printer(synop)

2:'eundf_a_'
1:'e554_a_'
1:eAnfang_a_
3:'e550_a_'
1:'e303_k_vom_afbruch_zur_hexe_bis_zur_ihren_ueberweltigung'
--------------------------------------------------------------------------------
8 'e550_a_'
--------------------------------------------------------------------------------
1:'e300_c_anfangssituation_ankunft_und_erkundigung_der_not'
2:eEnde_a_
1:'e300_e_vorfeld_des_kampfes_bis_zum_sieg'
3:'e550_a_'
1:'e303_o_erloesung_der_hexe'


